
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [1]:
using CSV, DataFrames, Statistics, Dates, Gadfly, GLM, Distributions, LinearAlgebra

# Chargement des données et nettoyage préliminaire

## Chargement des surverses

In [2]:
datainit = CSV.read("data/surverses.csv",missingstring="-99999")
first(data,5)

UndefVarError: UndefVarError: data not defined

## Nettoyage des données sur les surverses

#### Extraction des surverses pour les mois de mai à octobre inclusivement

In [3]:
datainit = filter(row -> month(row.DATE) > 4, datainit) 
datainit = filter(row -> month(row.DATE) < 11, datainit) 
first(data,5)

UndefVarError: UndefVarError: data not defined

#### Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"

In [4]:
raison = coalesce.(datainit[:,:RAISON],"Inconnue")
datainit[!,:RAISON] = raison
first(datainit,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = compacttype(::Type, ::Int64) at show.jl:39
└ @ DataFrames C:\Users\Clément\.julia\packages\DataFrames\XuYBH\src\abstractdataframe\show.jl:39


#### Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides

Ces facteurs correspondent à : 
- la fonte de neige (F), 
- les travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

In [5]:
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], datainit) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])
select!(datainit, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


#### Exclusion des lignes où :SURVERSE est manquante

In [6]:
surverse_df_init = dropmissing(datainit, disallowmissing=true)
surverse_df = dropmissing(data, disallowmissing=true)
first(surverse_df,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


## Chargement des précipitations

In [7]:
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


## Nettoyage des données sur les précipitations

#### Extraction des précipitations des mois de mai à octobre inclusivement

In [8]:
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing


# Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de voir s'il existe un lien entre les précipitations et les surverses.

Prenons arbitrairement l'ouvrage de débordement près du Bota-Bota (4350-01D). La station météorologique la plus proche est McTavish. Prenons deux variables explicatives simple :
- la somme journalière des précipitations
- le taux horaire maximum journalier de précipitations

#### Calcul de la quantité journalière de précipitations pour chacune des stations météorologiques

In [9]:
pcp_sum = by(data, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)
first(pcp_sum ,5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,missing
2,2013-05-02,0,0,0,0,missing
3,2013-05-03,0,0,0,0,missing
4,2013-05-04,0,0,0,0,missing
5,2013-05-05,0,0,0,0,missing


#### Extraction du taux horaire journalier maximum des précipitations pour chacune des stations météorologiques

In [10]:
pcp_max = by(data, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(pcp_max,5)

┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = groupreduce_init(::typeof(max), ::Nothing, ::Array{Union{Missing, Int64},1}, ::GroupedDataFrame{DataFrame}) at grouping.jl:560
└ @ DataFrames C:\Users\Clément\.julia\packages\DataFrames\XuYBH\src\groupeddataframe\grouping.jl:560


,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,missing
2,2013-05-02,0,0,0,0,missing
3,2013-05-03,0,0,0,0,missing
4,2013-05-04,0,0,0,0,missing
5,2013-05-05,0,0,0,0,missing


#### Inclusion dans un dataframe de ces deux variables explicatives potentielles

In [11]:
ouvrage1 = "4350-01D"
ouvrage2 = "3260-01D"
ouvrage3 = "3350-07D"
ouvrage4 = "4240-01D"
ouvrage5 = "4380-01D"
df0 = filter(row -> row.NO_OUVRAGE == ouvrage1, surverse_df_init)
df1 = filter(row -> row.NO_OUVRAGE == ouvrage1, surverse_df)
df2 = filter(row -> row.NO_OUVRAGE == ouvrage2, surverse_df)
df3 = filter(row -> row.NO_OUVRAGE == ouvrage3, surverse_df)
df4 = filter(row -> row.NO_OUVRAGE == ouvrage4, surverse_df)
df5 = filter(row -> row.NO_OUVRAGE == ouvrage5, surverse_df)

#print(size(df5,1))
n = size(df0,1) #max des size des dfs

x1 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x3 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x5 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x7 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x9 = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x2 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x4 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x6 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x8 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x10 = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier

for i=1:size(df0,1)
    
    ind1 = findfirst(pcp_sum[:,:date] .== df0[i,:DATE])
    ind2 = findfirst(pcp_max[:,:date] .== df0[i,:DATE])
    
    #McTavish
    x1[i] = pcp_sum[ind1,:McTavish]
    
    x2[i] = pcp_max[ind2,:McTavish]
    #Bellevue
    x3[i] = pcp_sum[ind1,:Bellevue]
    
    x4[i] = pcp_max[ind2,:Bellevue]
    #Assomption
    x5[i] = pcp_sum[ind1,:Assomption]
    
    x6[i] = pcp_max[ind2,:Assomption]
    #Trudeau
    x7[i] = pcp_sum[ind1,:Trudeau]
    
    x8[i] = pcp_max[ind2,:Trudeau]
    #StHubert
    x9[i] = pcp_sum[ind1,:StHubert]
    
    x10[i] = pcp_max[ind2,:StHubert]
    
end

In [14]:
X = [x1,x2,x3,x4,x5,x6,x7,x8,x9,x10]
for i=1:10
    xi = X[i]
    for j=1:size(xi,1)
        if isequal(missing, xi[j])
            s = 0
            c = 0
            if i%2 == 0
                for k=1:5
                    if isequal(missing, X[2*k][j])
                           s=s
                    else 
                        s += X[2*k][j]
                        c += 1
                    end
                end 
            else
                for k=1:5
                    if isequal(missing, X[2*(k-1) + 1][j])
                           s=s
                    else 
                        s += X[2*(k-1) + 1][j]
                        c += 1
                    end
                end
            end
            if c == 0
                xi[j] = 0
            else 
                xi[j] = floor(s/c)
            end
        end
    end
end

In [15]:
DF = [df1,df2,df3,df4,df5]
for df in DF
    x1c=copy(x1)
    x2c=copy(x2)
    x3c=copy(x3)
    x4c=copy(x4)
    x5c=copy(x5)
    x6c=copy(x6)
    x7c=copy(x7)
    x8c=copy(x8)
    x9c=copy(x9)
    x10c=copy(x10)
    Xc = [x1c,x2c,x3c,x4c,x5c,x6c,x7c,x8c,x9c,x10c]
    for j=1:size(df0,1)
        i = size(df0,1) + 1 - j
        date = df0[i,:DATE]
        if size(filter(row -> row.DATE == date, df),1)==0
            for xic in Xc
                deleteat!(xic, i)
            end
        end
    end
    df[!,:x1] = x1c
    df[!,:x2] = x2c
    df[!,:x3] = x3c
    df[!,:x4] = x4c
    df[!,:x5] = x5c
    df[!,:x6] = x6c
    df[!,:x7] = x7c
    df[!,:x8] = x8c
    df[!,:x9] = x9c
    df[!,:x10] = x10c
end

In [16]:
M1 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df1,  Bernoulli(), LogitLink())
M2 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df2,  Bernoulli(), LogitLink())
M3 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df3,  Bernoulli(), LogitLink())
M4 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df4,  Bernoulli(), LogitLink())
M5 = glm(@formula(SURVERSE ~ x1+x2+x3+x4+x5+x6+x7+x8+x9+x10), df5,  Bernoulli(), LogitLink())
print("")

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que la somme des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [17]:
#plot(df1, x=:SURVERSE, y=:x1, Geom.boxplot)

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que le maximum journalier des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [18]:
#plot(df, x=:SURVERSE, y=:MAX, Geom.boxplot)

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une probabilité de 1/2 sans considérer aucune variable explicative.

In [19]:
# Chargement du fichier de test
test = CSV.read("data/test.csv")

# Pour chacune des lignes du fichier test, comportant un ouvrage et une date, une prédiction est requise.
# Dans ce cas-ci, utilisons une prédiction les plus naîve. 
# On prédit avec une chance sur deux qu'il y ait surverse, sans utiliser de variables explicatives
n0 = size(test,1)

283

In [30]:
df1_ = filter(row -> row.NO_OUVRAGE == ouvrage1, test)
df2_ = filter(row -> row.NO_OUVRAGE == ouvrage2, test)
df3_ = filter(row -> row.NO_OUVRAGE == ouvrage3, test)
df4_ = filter(row -> row.NO_OUVRAGE == ouvrage4, test)
df5_ = filter(row -> row.NO_OUVRAGE == ouvrage5, test)

df0_ = copy(df1_[:DATE])
for df_ in [df2_, df3_, df4_, df5_]
    for i=1:size(df_,1)
        if (df_[i,:DATE] in df0_)==false
            push!(df0_, df_[i,:DATE])
        end
    end
end

n = length(df0_)

x1_ = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x3_ = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x5_ = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x7_ = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x9_ = Array{Union{Missing, Int64}}(missing, n) # variable pour la somme journalière
x2_ = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x4_ = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x6_ = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x8_ = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier
x10_ = Array{Union{Missing, Int64}}(missing, n) # variable pour le max journalier

for i=1:size(df0_,1)
    
    ind1 = findfirst(pcp_sum[:,:date] .== df0_[i])
    ind2 = findfirst(pcp_max[:,:date] .== df0_[i])
    
    #McTavish
    x1_[i] = pcp_sum[ind1,:McTavish]
    
    x2_[i] = pcp_max[ind2,:McTavish]
    #Bellevue
    x3_[i] = pcp_sum[ind1,:Bellevue]
    
    x4_[i] = pcp_max[ind2,:Bellevue]
    #Assomption
    x5_[i] = pcp_sum[ind1,:Assomption]
    
    x6_[i] = pcp_max[ind2,:Assomption]
    #Trudeau
    x7_[i] = pcp_sum[ind1,:Trudeau]
    
    x8_[i] = pcp_max[ind2,:Trudeau]
    #StHubert
    x9_[i] = pcp_sum[ind1,:StHubert]
    
    x10_[i] = pcp_max[ind2,:StHubert]
    
end

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[30]:6
└ @ Core In[30]:6


In [21]:
X = [x1_,x2_,x3_,x4_,x5_,x6_,x7_,x8_,x9_,x10_]
for xi_ in X
    for i=1:size(xi_,1)
        if isequal(missing, xi_[i])
            xi_[i] = 0
        end
    end
end

In [31]:
X = [x1_,x2_,x3_,x4_,x5_,x6_,x7_,x8_,x9_,x10_]
for i=1:10
    xi = X[i]
    for j=1:size(xi,1)
        if isequal(missing, xi[j])
            s = 0
            c = 0
            if i%2 == 0
                for k=1:5
                    if isequal(missing, X[2*k][j])
                           s=s
                    else 
                        s += X[2*k][j]
                        c += 1
                    end
                end 
            else
                for k=1:5
                    if isequal(missing, X[2*(k-1) + 1][j])
                           s=s
                    else 
                        s += X[2*(k-1) + 1][j]
                        c += 1
                    end
                end
            end
            if c == 0
                xi[j] = 0
            else 
                xi[j] = floor(s/c)
            end
        end
    end
end

In [32]:
df01 = copy(df0_)
df02 = copy(df0_)
df03 = copy(df0_)
df04 = copy(df0_)
df05 = copy(df0_)
DF0 = [df01,df02,df03,df04,df05]

DF = [df1_,df2_,df3_,df4_,df5_]
for k=1:5
    df_ = DF[k]
    x1_c=copy(x1_)
    x2_c=copy(x2_)
    x3_c=copy(x3_)
    x4_c=copy(x4_)
    x5_c=copy(x5_)
    x6_c=copy(x6_)
    x7_c=copy(x7_)
    x8_c=copy(x8_)
    x9_c=copy(x9_)
    x10_c=copy(x10_)
    X_c = [x1_c,x2_c,x3_c,x4_c,x5_c,x6_c,x7_c,x8_c,x9_c,x10_c]
    for j=1:length(df0_)
        i = length(df0_) + 1 - j
        date = df0_[i]
        if size(filter(row -> row.DATE == date, df_),1)==0
            for xi_c in X_c
                deleteat!(xi_c, i)
            end
            deleteat!(DF0[k],i)
        end
    end
    df_[!,:x0] = ones(Int64,length(x1_c))
    df_[!,:x1] = x1_c
    df_[!,:x2] = x2_c
    df_[!,:x3] = x3_c
    df_[!,:x4] = x4_c
    df_[!,:x5] = x5_c
    df_[!,:x6] = x6_c
    df_[!,:x7] = x7_c
    df_[!,:x8] = x8_c
    df_[!,:x9] = x9_c
    df_[!,:x10] = x10_c
end

In [33]:
select!(df1_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta1 = predict(M1,df1_) 
select!(df1_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta2 = predict(M2,df2_) 
select!(df1_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta3 = predict(M3,df3_) 
select!(df1_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta4 = predict(M4,df4_) 
select!(df1_, [:x0, :x1, :x2, :x3, :x4, :x5, :x6, :x7, :x8, :x9, :x10])
teta5 = predict(M5,df5_) 
print("")

In [34]:
Y1 = zeros(Int64,size(df1_,1))
Y2 = zeros(Int64,size(df2_,1))
Y3 = zeros(Int64,size(df3_,1))
Y4 = zeros(Int64,size(df4_,1))
Y5 = zeros(Int64,size(df5_,1))

Y1[teta1.>.5] .= 1
Y2[teta2.>.5] .= 1
Y3[teta3.>.5] .= 1
Y4[teta4.>.5] .= 1
Y5[teta5.>.5] .= 1
print("")

In [35]:
#Création de surverse test :
surverse = zeros(Int64, n0)
for i=1:n0
    ouvrage = test[i,:NO_OUVRAGE]
    date = test[i,:DATE]
    if ouvrage == ouvrage1
        ind = findfirst(df01[:] .== date)
        surverse[i] = Y1[ind]
    end
    if ouvrage == ouvrage2
        ind = findfirst(df02[:] .== date)
        surverse[i] = Y2[ind]
    end
    if ouvrage == ouvrage3
        ind = findfirst(df03[:] .== date)
        surverse[i] = Y3[ind]
    end
    if ouvrage == ouvrage4
        ind = findfirst(df04[:] .== date)
        surverse[i] = Y4[ind]
    end
    if ouvrage == ouvrage5
        ind = findfirst(df05[:] .== date)
        surverse[i] = Y5[ind]
    end
end

In [26]:
#print(test)

In [36]:
#surverse = rand(n0) .> .5

# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=surverse)
CSV.write("sampleSubmission.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.


"sampleSubmission.csv"